In [ ]:
import cx_Oracle
import pandas as pd

# 1. 오라클 DB 연결
dsn = cx_Oracle.makedsn("192.168.1.130", 1521, "FIMS2005")
db = cx_Oracle.connect("12512", "lma#12512", dsn) #아이디, 암호 수정
cursor = db.cursor()

print(db)

# 2. SQL 실행
sql = """
SELECT FUND_CD, ZEROIN_TYPE_CD, BM_CD1
FROM TMB002MC
WHERE ZEROIN_TYPE_GB = 'C2'
AND MEMB_CD= '1061'
AND END_YMD = '99999999'
AND ZEROIN_TYPE_CD IN ('A01A','A01G','B01A','B01F1','B01F2','B01G','B01F3', 'B01F4', 'B01F5') # B01F4, B01F5 추가
AND CLOSE_YMD IS NULL
ORDER BY ZEROIN_TYPE_CD
"""
cursor.execute(sql)


# 3. 결과를 DataFrame으로 변환
columns = [col[0] for col in cursor.description]
df_sql = pd.DataFrame(cursor.fetchall(), columns=columns)

cursor.close()
db.close()

df_sql.head()
df_sql.to_excel("sql 추출.xlsx", index = False, header=True)

# 4. 엑셀 파일 불러오기
excel_file = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\펀드 비중_사학연금.xlsx" #경로 및 파일명 수정
df_excel = pd.read_excel(excel_file, sheet_name='Sheet1')
df_excel.head()

# 5. FUND_CD 기준으로 BM_CD1와 ZEROIN_TYPE_CD 병합
df_merged = df_excel.merge(
    df_sql[['FUND_CD', 'BM_CD1', 'ZEROIN_TYPE_CD']],
    left_on='펀드코드',  # 엑셀 컬럼명
    right_on='FUND_CD',
    how='left'
)

# 6. 컬럼 업데이트
df_merged['BM코드'] = df_merged['BM_CD1']
df_merged['데이터구분'] = df_merged['ZEROIN_TYPE_CD']

# 7. 불필요한 컬럼 제거
df_merged = df_merged.drop(columns=['FUND_CD', 'BM_CD1', 'ZEROIN_TYPE_CD'])

# 8. 엑셀로 저장
df_merged.to_excel(
    r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\펀드 비중_사학연금_BM코드업데이트.xlsx",
    index=False
)


KeyboardInterrupt: 